In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Dense, Input, Flatten, concatenate,LSTM,Dropout
from tensorflow.keras.models import Model
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [ ]:
def ncf_model(arch):
  user_input = Input(shape=(1,), name = 'user_input')
  embed_user = Embedding(input_dim=nb_users, output_dim=64, input_length=1)(user_input)
  embed_user = Flatten()(embed_user)


  hotel_input = Input(shape=(1,), name = 'item_input')
  embed_hotel = Embedding(input_dim=nb_hotels, output_dim=64, input_length=1)(hotel_input)
  embed_hotel = Flatten()(embed_hotel)

  layers0 = concatenate([embed_user,embed_hotel])

  if len(arch)>1:

    hidden = Dense(arch[0],activation='relu')(layers0)

    for j in range(1,len(arch)):
      hidden = Dense(arch[j],activation='relu')(hidden)
  else:
    hidden = Dense(8,activation='relu')(layers0) 

  out = Dense(5)(hidden)
  model = Model(inputs=[user_input, hotel_input], 
                  outputs=out)
  
  model.compile(optimizer='adam',loss='mae',metrics=['mae'])

  return model

In [ ]:
df = pd.DataFrame(data={'alpha':[],'beta':[],	'mean mae':[],	'mae_cleanliness':[],
                        'mae_location':[],'mae_rooms':[],'mae_service':[],'mae_sleep':[],'mae_value':[],	
                        'mean rmse':[], 'rmse_cleanliness':[],'rmse_location':[],'rmse_rooms':[],	
                        'rmse_service':[],'rmse_sleep':[],'rmse_value':[]})
df

,alpha,beta,mean mae,mae_cleanliness,mae_location,mae_rooms,mae_service,mae_sleep,mae_value,mean rmse,rmse_cleanliness,rmse_location,rmse_rooms,rmse_service,rmse_sleep,rmse_value


In [ ]:
def agg_():
  inp_agg = Input(shape=(5,), name = 'creteria_input')
  agg = Dense(64,activation='relu')(inp_agg)
  agg = Dropout(0.2)(agg)
  agg = Dense(128,activation='relu')(inp_agg)
  agg = Dropout(0.3)(agg)
  agg = Dense(64,activation='relu')(inp_agg)
  agg = Dropout(0.2)(agg)
  agg = Dense(32,activation='relu')(agg)
  agg = Dropout(0.2)(agg)

  out= Dense(1,activation='relu')(agg)

  agg_model = Model(inputs=inp_agg, 
                    outputs=out)

  agg_model.compile(optimizer='adam',loss='mae',metrics=['mae'])
  
  return agg_model

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.neighbors import DistanceMetric
kf = KFold(n_splits=5,random_state=24,shuffle=True)



dist = DistanceMetric.get_metric('hamming')




value_alpha = [0.7,0.5,0.3]
dic_alpha = {}
for alpha in value_alpha:

        dic_alpha[alpha] = {}
        dic_alpha[alpha]['mae_clean'] = []
        dic_alpha[alpha]['mae_location'] = []
        dic_alpha[alpha]['mae_rooms'] = []
        dic_alpha[alpha]['mae_service'] = []
        dic_alpha[alpha]['mae_value'] = []
        dic_alpha[alpha]['mae_mean'] = []

        dic_alpha[alpha]['rmse_clean'] = []
        dic_alpha[alpha]['rmse_location'] = []
        dic_alpha[alpha]['rmse_rooms'] = []
        dic_alpha[alpha]['rmse_service'] = []
        dic_alpha[alpha]['rmse_value'] = []
        dic_alpha[alpha]['rmse_mean'] = []


        dic_alpha[alpha]['mae_agg'] = []
        dic_alpha[alpha]['rmse_agg'] = []
k=7
for i in range(1,6):
    pred_Bert_cf = []
    train_data = pd.read_csv('/content/drive/MyDrive/fold_with_sentiment'+str(i)+'_train.csv')
    test_data = pd.read_csv('/content/drive/MyDrive/fold_with_sentiment'+str(i)+'_test.csv')
    train_data.shape , test_data.shape
    nb_users = int(max(train_data.user_id.unique())+1)
    nb_hotels = int(max(train_data.offering_id.unique())+1)
    nb_users, nb_hotels

    ncf = ncf_model([128,64,32,16,8])

    y = train_data[['cleanliness','location','rooms','service','value']].values
    y_test = test_data[['cleanliness','location','rooms','service','value']].values
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
    ncf.fit([train_data.user_id.values,train_data.offering_id.values],y,validation_split=0.1,callbacks=[callback],
              
              epochs=20,batch_size=128)
    ncf_pred = ncf.predict([test_data.user_id.values,test_data.offering_id.values])
    cmp=0
    for idx,row in test_data.iterrows():
      cmp+=1
      precedent_data = train_data[train_data['user_id']==row['user_id']]
      precedent_hotels = precedent_data.offering_id.values
      prcd_reviews = precedent_data.text.values
      hotel_id = row['offering_id']
      
      if hotel_id in train_data.offering_id.values:
        precedent_data = train_data[train_data['user_id']==row['user_id']]

        if precedent_data.shape[0] > 0:

          precedent_hotels = precedent_data.offering_id.values
          prcd_reviews = precedent_data.sentiment_score.values
          
        
          prcedent_data_sentiment_review = {}
          for h_id,rev in zip(precedent_hotels,prcd_reviews):
            
            
            prcedent_data_sentiment_review[h_id] = rev
            
          
          user = row['user_id']

          maybe_sim_users = []
          sim = {}
          if cmp%100==0:
            print('rest',test_data.shape[0] - cmp)

          users = train_data[train_data['offering_id']==hotel_id].user_id.unique()
          for user1 in users:
            if (user1 != user):
              

              user1_data = train_data[train_data['user_id']==user1]
              sim_items = set(precedent_hotels).intersection(set(user1_data.offering_id.values))
              if len(sim_items) > 1:
                  
                  maybe_sim_users.append(user1)
                
                  user_rat = []
                  user1_rat = []
                  for item in sim_items:
                    
                    user_rat.append(prcedent_data_sentiment_review[item])

                  for item in sim_items:
                    rev = user1_data[user1_data['offering_id']==item].sentiment_score.values
                    
                    
                    user1_rat.append(rev[0])
                  
                  if (1 - dist.pairwise([user_rat],[user1_rat])[0][0])> 0.5:
                    sim[user1] = 1 - dist.pairwise([user_rat],[user1_rat])[0][0]
          
          if len(sim.keys())>0: #similar users
            rat = [0.0,0.0,0.0,0.0,0.0]
            sum_sim = 0.0
            for user1 in list(sim.keys())[:k]:
              rat += sim[user1] * train_data[(train_data['user_id']==user1) & (train_data['offering_id']==hotel_id ) ][['cleanliness','location','rooms','service','value']].values[0]
              sum_sim+= sim[user1]
            rat = rat / sum_sim
          
          
          else : #no similar users
            rat = [0.0,0.0,0.0,0.0,0.0]
            for i,col in zip([0,1,2,3,4],['cleanliness','location','rooms','service','value']):
              rat[i] = np.mean(train_data[train_data['offering_id']==hotel_id][[col]].values)
          
        else: #new_user
            rat = [0.0,0.0,0.0,0.0,0.0]
            for i,col in zip([0,1,2,3,4],['cleanliness','location','rooms','service','value']):
              rat[i] = np.mean(train_data[train_data['offering_id']==hotel_id][[col]].values)  

      elif train_data[train_data['user_id']==row['user_id']].shape[0]>0: #new_item
        rat = [0.0,0.0,0.0,0.0,0.0]
        for i,col in zip([0,1,2,3,4],['cleanliness','location','rooms','service','value']):
            rat[i] = np.mean(precedent_data[[col]].values)
      else: #new_user , new_item
        #rnd_petubation = random.uniform(-1,1)
        #val = rnd_petubation + 3.0
        rat = [3.0,3.0,3.0,3.0,3.0]
      pred_Bert_cf.append(rat.copy())

    pred_Bert_cf = np.array(pred_Bert_cf)

    
    over_train = train_data.overall.values
    over_test = test_data.overall.values

    agg_model = agg_()

    #agg_train_inp = model.predict([X_user_train,X_hotel_train])
    print('\n train agg model \n')
    agg_model.fit(y,over_train,validation_split=0.1,epochs=20,batch_size=64)

    for alpha in value_alpha:
      pred = alpha*ncf_pred + (1-alpha)*pred_Bert_cf
      pred = np.array(pred)
      dic_alpha[alpha]['mae_clean'].append(mae(pred[:,0],y_test[:,0]))
      dic_alpha[alpha]['mae_location'].append(mae(pred[:,1],y_test[:,1]))
      dic_alpha[alpha]['mae_rooms'].append(mae(pred[:,2],y_test[:,2]))
      dic_alpha[alpha]['mae_service'].append(mae(pred[:,3],y_test[:,3]))
      dic_alpha[alpha]['mae_value'].append(mae(pred[:,4],y_test[:,4]))
      dic_alpha[alpha]['mae_mean'].append(mae(pred,y_test))

      dic_alpha[alpha]['rmse_clean'].append(mse(pred[:,0],y_test[:,0],squared=False))
      dic_alpha[alpha]['rmse_location'].append(mse(pred[:,1],y_test[:,1],squared=False))
      dic_alpha[alpha]['rmse_rooms'].append(mse(pred[:,2],y_test[:,2],squared=False))
      dic_alpha[alpha]['rmse_service'].append(mse(pred[:,3],y_test[:,3],squared=False))
      dic_alpha[alpha]['rmse_value'].append(mse(pred[:,4],y_test[:,4],squared=False))
      dic_alpha[alpha]['rmse_mean'].append(mse(pred,y_test,squared=False))

      

      dic_alpha[alpha]['mae_agg'].append(mae(agg_model.predict(pred),over_test))
      dic_alpha[alpha]['rmse_agg'].append(mse(agg_model.predict(pred),over_test,squared=False))
      

 
      
    


    




/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_distance_metric.py:14: FutureWarning: sklearn.neighbors.DistanceMetric has been moved to sklearn.metrics.DistanceMetric in 1.0. This import path will be removed in 1.3
  category=FutureWarning,


Epoch 1/20
55/55 [==============================] - 4s 8ms/step - loss: 2.9664 - mae: 2.9664 - val_loss: 1.8841 - val_mae: 1.8841
Epoch 2/20
55/55 [==============================] - 0s 5ms/step - loss: 1.3060 - mae: 1.3060 - val_loss: 1.0716 - val_mae: 1.0716
Epoch 3/20
55/55 [==============================] - 0s 5ms/step - loss: 0.7651 - mae: 0.7651 - val_loss: 0.8849 - val_mae: 0.8849
Epoch 4/20
55/55 [==============================] - 0s 5ms/step - loss: 0.6881 - mae: 0.6881 - val_loss: 0.8817 - val_mae: 0.8817
Epoch 5/20
55/55 [==============================] - 0s 5ms/step - loss: 0.6615 - mae: 0.6615 - val_loss: 0.8899 - val_mae: 0.8899
Epoch 6/20
55/55 [==============================] - 0s 5ms/step - loss: 0.6417 - mae: 0.6417 - val_loss: 0.8750 - val_mae: 0.8750
Epoch 7/20
55/55 [==============================] - 0s 5ms/step - loss: 0.6203 - mae: 0.6203 - val_loss: 0.8752 - val_mae: 0.8752
Epoch 8/20
55/55 [==============================] - 0s 5ms/step - loss: 0.5964 - mae: 0.59

In [ ]:
dic_alpha

{0.3: {'mae_agg': [0.8589521718616939,
   0.898751452765519,
   0.8552659762312486,
   0.8627633862216672,
   0.9080369353479384],
  'mae_clean': [0.7588111626844196,
   0.7596641039035698,
   0.7348558723202698,
   0.7268705110067487,
   0.7863926434196293],
  'mae_location': [0.6460112883560403,
   0.6452209277525235,
   0.6168829736752834,
   0.6503681020937337,
   0.6469980304031134],
  'mae_mean': [0.8155875476551735,
   0.8054119883780577,
   0.8059292984105317,
   0.803803055389228,
   0.8312221651076314],
  'mae_rooms': [0.8536924001572616,
   0.8441974747712361,
   0.8683485468475676,
   0.8555300116869902,
   0.8544731436438205],
  'mae_service': [0.8694687956099406,
   0.8578220481172657,
   0.868837307405532,
   0.8532057094935563,
   0.901113805005954],
  'mae_value': [0.9499540914682013,
   0.9201553873456898,
   0.9407217918040045,
   0.9330409426651076,
   0.967133203065635],
  'rmse_agg': [1.0851389500289321,
   1.1223809614859788,
   1.1185759883354152,
   1.103389261

In [ ]:
dic_final = {}
dic_agg = {}
for alpha in value_alpha:
  dic_final[alpha] = {}
  dic_agg[alpha] = {}
  for key in dic_alpha[alpha].keys():
    if key in ['mae_agg','rmse_agg']:
      dic_agg[alpha][key] = np.mean(dic_alpha[alpha][key])
      
    dic_final[alpha][key] = np.mean(dic_alpha[alpha][key])

In [ ]:
dic_final

{0.3: {'mae_agg': 0.8767539844856135,
  'mae_clean': 0.7533188586669275,
  'mae_location': 0.6410962644561389,
  'mae_mean': 0.8123908109881246,
  'mae_rooms': 0.8552483154213751,
  'mae_service': 0.8700895331264498,
  'mae_value': 0.9422010832697276,
  'rmse_agg': 1.1135673721599255,
  'rmse_clean': 1.040141321643194,
  'rmse_location': 0.898038524051897,
  'rmse_mean': 1.091285790597597,
  'rmse_rooms': 1.1204432813598633,
  'rmse_service': 1.1826425018071849,
  'rmse_value': 1.2151633241258462},
 0.5: {'mae_agg': 0.8700886352773163,
  'mae_clean': 0.7541004164652809,
  'mae_location': 0.650020294131657,
  'mae_mean': 0.8123932597114273,
  'mae_rooms': 0.8552114614185902,
  'mae_service': 0.8660746062178672,
  'mae_value': 0.9365595203237413,
  'rmse_agg': 1.1095233170397136,
  'rmse_clean': 1.0413771642306586,
  'rmse_location': 0.90249556193386,
  'rmse_mean': 1.091713193238128,
  'rmse_rooms': 1.122229038611068,
  'rmse_service': 1.1802382906448474,
  'rmse_value': 1.2122259107702

In [ ]:
dic_agg

{0.3: {'mae_agg': 0.8767539844856135, 'rmse_agg': 1.1135673721599255},
 0.5: {'mae_agg': 0.8700886352773163, 'rmse_agg': 1.1095233170397136},
 0.7: {'mae_agg': 0.8696121271439488, 'rmse_agg': 1.1141512520094656}}

In [ ]:
import json
with open("/content/drive/MyDrive/finally_NCF+BERT_cf_with_callback.json", "w") as outfile:
    json.dump(dic_final, outfile)

#Alpha 0.7 beta 0.3

In [ ]:
pred = 0.7*ncf_pred + 0.3*pred_Bert_cf
mae(pred,y_test)

0.8222287391593394

#Alpha 0.5 beta 0.5

In [ ]:
pred = 0.5*ncf_pred + 0.5*pred_Bert_cf
mae(pred,y_test)

0.8178455189825333

#Alpha 0.3 beta 0.7

In [ ]:
pred = 0.3*ncf_pred + 0.7*pred_Bert_cf
mae(pred,y_test)

0.8234194200785353